In [22]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import pyproj
from shapely.ops import transform
from shapely.geometry import Point
import pickle
import numpy as np

In [21]:
with open("../processed_data/bkk_augmented_graph.pickle", "rb") as f:
    G = pickle.load(f)

In [24]:
hospitals = pd.read_csv('../processed_data/depot_locations.csv')

In [ ]:
fuel = ox.features_from_place("Bangkok, Thailand", tags={"amenity": "fuel"})
fuel = fuel.dropna(how='any', subset='name')
fuel = fuel.to_crs(32647)
fuel["y"] = fuel.geometry.centroid.y
fuel["x"] = fuel.geometry.centroid.x
fuel["geometry"] = fuel.apply(lambda row: Point(row["x"], row["y"]), axis=1)

fuel = fuel[['geometry', 'name']]
fuel = fuel.to_crs(4326)

fuel["lat"] = fuel.geometry.y
fuel["lon"] = fuel.geometry.x


/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


In [ ]:
fuel.to_csv('gas_stations.csv')

In [41]:
hospitals_df = pd.read_csv('../processed_data/depot_locations.csv')

In [42]:
# Convert stations
stations_gdf = gpd.GeoDataFrame(
    fuel,
    geometry=gpd.points_from_xy(fuel.lon, fuel.lat),
    crs="EPSG:4326"
)

# Convert hospitals
hospitals_gdf = gpd.GeoDataFrame(
    hospitals_df,
    geometry=gpd.points_from_xy(hospitals_df.lon, hospitals_df.lat),
    crs="EPSG:4326"
)

In [ ]:
stations_gdf = stations_gdf.to_crs(32647)
hospitals_gdf = hospitals_gdf.to_crs(32647)


In [44]:
nearest = gpd.sjoin_nearest(
    stations_gdf,
    hospitals_gdf,
    how="left",
    distance_col="dist_m"
)


In [64]:
filtered = nearest[nearest["dist_m"] > 3000].drop(["lat_left", "lon_left", "index_right", "name:en", "lon_right", "lat_right", "dist_m"], axis=1)

In [65]:
filtered["lon"] = filtered.geometry.x
filtered["lat"] = filtered.geometry.y
filtered

geometry  \
element id                                           
node    357091596     POINT (674187.65 1513220.23)   
        379014542    POINT (680640.88 1514074.392)   
        1157495593  POINT (653698.927 1514889.796)   
        1629229640  POINT (652455.059 1516213.028)   
        1629229645  POINT (652523.285 1516150.037)   
...                                            ...   
way     1333533898  POINT (661004.264 1509618.799)   
        1352044853    POINT (667388.37 1512828.81)   
        1387528807  POINT (677780.756 1533880.941)   
        1387773032  POINT (684468.057 1533620.677)   
        1444851071  POINT (666795.485 1512218.058)   

                                               name            lon  \
element id                                                           
node    357091596                          สยามแก๊ส  674187.649803   
        379014542                             Susco  680640.880192   
        1157495593                           ป.ต.ท.  653698.927303   
        1629229640              ปั๊มน้ำมันคาลเท็กซ์  652455.059169   
        1629229645  ปั๊มน้ำมันเชลล์ หจก.สุกิจบริการ  652523.284684   
...                                             ...            ...   
way     1333533898                            Susco  661004.263768   
        1352044853                           ซัสโก้  667388.369507   
        1387528807                               PT  677780.756450   
        1387773032                           ซัสไก้  684468.057173   
        1444851071                         สยามแก๊ส  666795.485336   

                             lat  
element id                        
node    357091596   1.513220e+06  
        379014542   1.514074e+06  
        1157495593  1.514890e+06  
        1629229640  1.516213e+06  
        1629229645  1.516150e+06  
...                          ...  
way     1333533898  1.509619e+06  
        1352044853  1.512829e+06  
        1387528807  1.533881e+06  
        1387773032  1.533621e+06  
        1444851071  1.512218e+06  

[94 rows x 4 columns]

In [66]:
filtered_nodes = ox.distance.nearest_nodes(
    G,
    X=filtered.to_crs(4326)["lon"],
    Y=filtered.to_crs(4326)["lat"]
)

filtered_nodes = list(set(filtered_nodes)) 

In [67]:
len(filtered_nodes)

82

In [68]:
with open("gas_stations.pickle", "wb") as f:
    pickle.dump(filtered_nodes, f)

In [5]:
tags = {
    "amenity": [
        # "hospital",
        # "fire_station",
        # "police",
        # "college",
        # "university",
        "fuel"
    ],
    # "shop": "mall",
    # "railway": "station"
}

# Download everything in one pass
print("downloading POIs")
pois = ox.features_from_place("Bangkok, Thailand", tags=tags)
print(len(pois))
# Compute lat/lon of centroids
print("computing centroids")
pois = pois.to_crs(32647)
pois["y"] = pois.geometry.centroid.y
pois["x"] = pois.geometry.centroid.x


downloading POIs
493
computing centroids


/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/predicates.py:878: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
/opt/anaconda3/envs/311_final/lib/python3.10/site-packages/shapely/set_operations.py:451: RuntimeWarning: invalid value encountered in union
  return lib.union(a, b, **kwargs)


In [6]:
pois["geometry"] = pois.apply(lambda row: Point(row["x"], row["y"]), axis=1)

pois = pois[['geometry', 'name']]

In [7]:
pois

geometry       name
element id                                                   
node    292743083    POINT (669483.488 1517152.14)      Shell
        357091596     POINT (674187.65 1513220.23)   สยามแก๊ส
        357091616   POINT (676081.989 1511779.979)   สยามแก๊ส
        357091622    POINT (676526.34 1511702.235)       Esso
        379014542    POINT (680640.88 1514074.392)      Susco
...                                            ...        ...
way     1444580022  POINT (661255.957 1517225.652)     Caltex
        1444851071  POINT (666795.485 1512218.058)   สยามแก๊ส
        1448269780   POINT (657850.267 1504639.74)        NaN
        1452541928  POINT (666947.391 1523682.669)  คาลเท็กซ์
        1455437634  POINT (667362.941 1527114.639)        PTT

[493 rows x 2 columns]

In [8]:
pois_ll = pois.to_crs("EPSG:4326")
pois_ll = pois_ll.dropna(axis=0, how='any', subset=['name'])

# extract columns
pois_ll["lon"] = pois_ll.geometry.x
pois_ll["lat"] = pois_ll.geometry.y

In [9]:
pois_ll

geometry                     name  \
element id                                                                
node    292743083   POINT (100.56736 13.71857)                    Shell   
        357091596   POINT (100.61061 13.68275)                 สยามแก๊ส   
        357091616   POINT (100.62803 13.66962)                 สยามแก๊ส   
        357091622   POINT (100.63214 13.66889)                     Esso   
        379014542   POINT (100.67032 13.69008)                    Susco   
...                                        ...                      ...   
way     1436318817  POINT (100.57298 13.88951)  ปตท.แจ้งวัฒนะ (หลักสี่)   
        1444580022   POINT (100.4913 13.71971)                   Caltex   
        1444851071  POINT (100.54222 13.67413)                 สยามแก๊ส   
        1452541928   POINT (100.5443 13.77775)                คาลเท็กซ์   
        1455437634  POINT (100.54835 13.80875)                      PTT   

                           lon        lat  
element id                                 
node    292743083   100.567362  13.718573  
        357091596   100.610612  13.682755  
        357091616   100.628033  13.669622  
        357091622   100.632135  13.668893  
        379014542   100.670318  13.690080  
...                        ...        ...  
way     1436318817  100.572981  13.889509  
        1444580022  100.491298  13.719709  
        1444851071  100.542219  13.674131  
        1452541928  100.544302  13.777750  
        1455437634  100.548349  13.808747  

[379 rows x 4 columns]

In [10]:
pois_ll.to_csv("../processed_data/depot_locations2.csv", index=False)

In [11]:
import osmnx as ox
import folium


In [12]:
center_lat = pois_ll.geometry.y.mean()
center_lon = pois_ll.geometry.x.mean()

m = folium.Map(location=[center_lat, center_lon], zoom_start=14)


In [13]:
pois_ll

geometry                     name  \
element id                                                                
node    292743083   POINT (100.56736 13.71857)                    Shell   
        357091596   POINT (100.61061 13.68275)                 สยามแก๊ส   
        357091616   POINT (100.62803 13.66962)                 สยามแก๊ส   
        357091622   POINT (100.63214 13.66889)                     Esso   
        379014542   POINT (100.67032 13.69008)                    Susco   
...                                        ...                      ...   
way     1436318817  POINT (100.57298 13.88951)  ปตท.แจ้งวัฒนะ (หลักสี่)   
        1444580022   POINT (100.4913 13.71971)                   Caltex   
        1444851071  POINT (100.54222 13.67413)                 สยามแก๊ส   
        1452541928   POINT (100.5443 13.77775)                คาลเท็กซ์   
        1455437634  POINT (100.54835 13.80875)                      PTT   

                           lon        lat  
element id                                 
node    292743083   100.567362  13.718573  
        357091596   100.610612  13.682755  
        357091616   100.628033  13.669622  
        357091622   100.632135  13.668893  
        379014542   100.670318  13.690080  
...                        ...        ...  
way     1436318817  100.572981  13.889509  
        1444580022  100.491298  13.719709  
        1444851071  100.542219  13.674131  
        1452541928  100.544302  13.777750  
        1455437634  100.548349  13.808747  

[379 rows x 4 columns]

In [14]:
folium.GeoJson(
    pois_ll.to_json(),
    name="nodes",
    marker=folium.CircleMarker(radius=3, color="red"),
).add_to(m)


In [15]:
m


In [ ]:
m.save("bangkok_hospitals_map.html")
